In [ ]:
import jax
from jax import grad, jit
import jax.numpy as jnp

In [ ]:
@jit
def f(x):
  for i in range(3):
    x = 2 * x
  return x

print(f(3))

In [16]:
@jit
def g(x):
  y = 0.
  for i in range(x.shape[0]):
    y = y + x[i]
  return y

print(g(jnp.array([1., 2., 3.])))

# jax.make_jaxpr(g)(jnp.array([1., 2., 3., 4.2]))
# 运行上面的代码可以看到，如果 x 的长度不一样，编译的结果也不一样
# 对于不同长度的输入，jax 会重新编译

6.0


In [ ]:
# static_argnames 的一个例子

def f(x):
  if x < 3:
    return 3. * x ** 2
  else:
    return -4 * x

f = jit(f, static_argnames='x')

print(f(2.))

In [ ]:
# static_argnames 的另一个例子

def f(x, n):
  y = 0.
  for i in range(n):
    y = y + x[i]
  return y

f = jit(f, static_argnames='n')

f(jnp.array([2., 3., 4.]), 2)

In [ ]:
# static_argnames 的更多例子
# static_argnames 可以指定多个参数为静态参数
# 或者可以使用 static_argnums 来指定参数的位置

def example_fun(length, val):
  return jnp.ones((length,)) * val
# un-jit'd works fine
print(example_fun(5, 4))

# 注意，这个函数 example_fun 不能够直接 jit，因为它的第一个参数是一个变量

# static_argnames tells JAX to recompile on changes at these argument positions:
good_example_jit = jit(example_fun, static_argnames='length')
# first compile
print(good_example_jit(10, 4))
# recompiles
print(good_example_jit(5, 8))

# 如果想指定更多的参数为静态参数，可以使用 static_argnames
more_example_jit = jit(example_fun, static_argnames=['length', 'val'])
# 或者换个写法
more_example_jit = jit(example_fun, static_argnums=(0, 1))
# first compile
print(more_example_jit(10, 4))
# recompiles
print(more_example_jit(5, 8))
# 这里的 length 和 val 都是静态参数
# 但是如果 length 和 val 都是动态参数，那么就会报错
print(more_example_jit(12, 8)) # 这里的 length 和 val 都是动态参数

In [ ]:
# 如果需要在 jit 版本的函数中使用 print 语句，可以使用 jax.debug.print

import jax

@jit
def f(x):
    jax.debug.print("The first printed number is {x}", x=x)
    y = 2 * x
    print(y)
    jax.debug.print("The second printed number is {}", y)
    return y
f(2)